In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from selenium.common.exceptions import TimeoutException, WebDriverException, ElementClickInterceptedException, NoSuchElementException
import numpy as np
from multiprocessing import Pool

In [2]:
!pip install selenium

     |████████████████████████████████| 911kB 2.7MB/s 


In [0]:
url = 'https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm'
r = requests.get(url)
html_doc = r.text

In [0]:
#creation of a BeautifulSoup object
soup = BeautifulSoup(html_doc, 'html.parser')
soup

ï»¿<!DOCTYPE doctype html>

<html class="no-js" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0 maximum-scale=1.6, user-scalable=1" name="viewport"/>
<meta content="Board of Governors of the Federal Reserve System, Federal Reserve Board of Governors, Federal Reserve Board, Federal Reserve" name="keywords"/>
<meta content="The Federal Reserve Board of Governors in Washington DC." name="description"/>
<meta content="Board of Governors of the Federal Reserve System" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="" property="og:image"/>
<meta content="summary" name="twitter:card"/>
<meta content="" name="twitter:image"/>
<title>The Fed - Meeting calendars and information</title>
<link href="/css/bootstrap.css" rel="stylesheet" type="text/css"/>
<link href="/css/bluesteel-theme.css" rel="stylesheet" type="text/css"/>
<script src="/js/

In [0]:
# <div class="col-xs-12 col-sm-8 col-md-9" id="article">

In [0]:
items = soup.find('div',{"class":"col-xs-12 col-sm-8 col-md-9", "id":"article"})
items

<div class="col-xs-12 col-sm-8 col-md-9" id="article">
<h3>Minutes of the Federal Open Market Committee</h3>
<!-- Article Area -->
<p><strong>January 29-30, 2019</strong></p>
<p>A joint meeting of the Federal Open Market Committee and the Board of Governors was held in the offices of the Board of Governors of the Federal Reserve System in Washington, D.C., on Tuesday, January 29, 2019, at 10:00 a.m. and continued on Wednesday, January 30, 2019, at 9:00 a.m.<a href="#fn1" title="footnote 1"><sup>1</sup></a><a name="f1"></a></p>
<p>PRESENT:<br/>
Jerome H. Powell, Chairman<br/>
John C. Williams, Vice Chairman<br/>
Michelle W. Bowman<br/>
Lael Brainard<br/>
James Bullard<br/>
Richard H. Clarida<br/>
Charles L. Evans<br/>
Esther L. George<br/>
Randal K. Quarles<br/>
Eric Rosengren</p>
<p>Patrick Harker, Robert S. Kaplan, Neel Kashkari, Loretta J. Mester, and Michael Strine, Alternate Members of the Federal Open Market Committee</p>
<p>Thomas I. Barkin, Raphael W. Bostic, and Mary C. Daly, P

In [0]:
driver = webdriver.Chrome('/Users/ewelinka/Desktop/STUDIA/MGR_Data Science/SEMESTR 2/Webscraping/Project/chromedriver')
driver.get('https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm')

In [0]:
import requests
from bs4 import BeautifulSoup
import os
import re
from multiprocessing import Pool

In [0]:
# SCRAPING

def minarchive(year):
    url_main = 'https://www.federalreserve.gov'
    year = int(year)
    
    if year > 2012: #Post 2012 has a different URL
        url = url_main+'/monetarypolicy/fomccalendars.htm'
    else:
        url = url_main + '/monetarypolicy/fomchistorical'+str(year)+'.htm'
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    min_link = soup.findAll('a',href=re.compile('(?=.*minutes)(?=.*'+str(year)+')'), 
                                text=lambda text: text and 'PDF' not in text)
    
    #Methods
    m0={'name':'div','attrs':{'id':'article'}}
    m1={'name':'div','attrs':{'id':'leftText'}}
    m2={'name':'p'}
    
    for m in min_link:
        url = str(m['href']) if 'federalreserve.gov' in str(m['href']) else url_main + str(m['href'])
        filename = re.search("(\d{4})(\d{2})(\d{2})",url)[0]
        soup = BeautifulSoup(requests.get(url).content,'lxml')
        
        i=0
        
        main_text=[]
        while not main_text and i<3:
            main_text = soup.findAll(**eval('m'+str(i)))
            i += 1        
        saveFile(filename,year,main_text)


def saveFile(fname,year,text):
    main_directory = '/Users/ewelinka/Desktop/fomcc'
    
    os.chdir(main_directory)
    directory = main_directory + str(year) + '/'
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        #check if file name already exists
    if not os.path.isfile(fname):
        os.chdir(directory)
        file= open(fname + '.txt', 'w')
        
        text_clean = '\n'.join(t.text for t in text)
        text_clean = re.sub(r'\r\n',' ',text_clean)
        text_clean = re.sub(r'(?<!\n)\n(?!\n)',r'\n\n',text_clean)
        
        file.write(text_clean)
        file.close()        
    

if __name__ == '__main__':
    start_year,end_year = 2019,2004
    p = Pool(processes=(start_year-end_year))
    p.map(minarchive,range(start_year,end_year -1,-1))
    p.close()

**to do: FOMC 2013**

In [0]:
def minarchive(year):
    
    url = 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180131a.htm'
    #url_main = 'https://www.federalreserve.gov'
    year = int(year)
    
    #if year > 2012: #Post 2012 has a different URL
    #    url = url_main+'/monetarypolicy/fomccalendars.htm'
    #else:
    #    url = url_main + '/monetarypolicy/fomchistorical'+str(year)+'.htm'
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    min_link = soup.findAll('a',href=re.compile('(?=.*minutes)(?=.*'+str(year)+')'), 
                                text=lambda text: text and 'PDF' not in text)
    
    #Methods
    m0={'name':'div','attrs':{'id':'article'}}
    m1={'name':'div','attrs':{'id':'leftText'}}
    m2={'name':'p'}
    
    for m in min_link:
        url = str(m['href']) if 'federalreserve.gov' in str(m['href']) else url_main + str(m['href'])
        filename = re.search("(\d{4})(\d{2})(\d{2})",url)[0]
        soup = BeautifulSoup(requests.get(url).content,'lxml')
        
        i=0
        
        main_text=[]
        while not main_text and i<3:
            main_text = soup.findAll(**eval('m'+str(i)))
            i += 1        
        saveFile(filename,year,main_text)


def saveFile(fname,year,text):
    main_directory = '/Users/ewelinka/Desktop/'
    
    os.chdir(main_directory)
    directory = main_directory + str(year) + '/'
    
    if not os.path.exists(directory):
        os.makedirs(directory)
        #check if file name already exists
    if not os.path.isfile(fname):
        os.chdir(directory)
        file= open(fname + '.txt', 'w')
        
        text_clean = '\n'.join(t.text for t in text)
        text_clean = re.sub(r'\r\n',' ',text_clean)
        text_clean = re.sub(r'(?<!\n)\n(?!\n)',r'\n\n',text_clean)
        
        file.write(text_clean)
        file.close()        
    

if __name__ == '__main__':
    start_year,end_year = 2018,2004
    p = Pool(processes=(start_year-end_year))
    p.map(minarchive,range(start_year,end_year -1,-1))
    p.close()

In [0]:
def minarchive(year):
    
    #url = 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180131a.htm'
    url_main = 'https://www.federalreserve.gov/newsevents/pressreleases/monetary'
    year = int(year)
    
    #if year > 2012: #Post 2012 has a different URL
    #    url = url_main+str(year)+'a.htm'
    #else:
    url = url_main + '/monetarypolicy/fomchistorical'+str(year)+'.htm'
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    min_link = soup.findAll('a',href=re.compile('(?=.*minutes)(?=.*'+str(year)+')'), 
                                text=lambda text: text and 'PDF' not in text)
    
    #Methods
    m0={'name':'div','attrs':{'id':'article'}}
    m1={'name':'div','attrs':{'id':'leftText'}}
    m2={'name':'p'}
    
    for m in min_link:
        url = str(m['href']) if 'federalreserve.gov' in str(m['href']) else url_main + str(m['href'])
        filename = re.search("(\d{4})(\d{2})(\d{2})",url)[0]
        soup = BeautifulSoup(requests.get(url).content,'lxml')
        
        i=0
        
        main_text=[]
        while not main_text and i<3:
            main_text = soup.findAll(**eval('m'+str(i)))
            i += 1        
        saveFile(filename,year,main_text)


In [0]:
def minarchive(year):
    
    #url = 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20180131a.htm'
    url_main = 'https://www.federalreserve.gov/newsevents/pressreleases/monetary'
    year = int(year)
    
    #if year > 2012: #Post 2012 has a different URL
    #    url = url_main+str(year)+'a.htm'
    #else:
    url = url_main + '/monetarypolicy/fomchistorical'+str(year)+'.htm'
    
    r = requests.get(url)
    soup = BeautifulSoup(r.text)